In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis
from regions import CircleSkyRegion
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    GaussianSpatialModel,
    SkyModel,
    Models,
    PointSpatialModel,
    FoVBackgroundModel,
    ExpCutoffPowerLawSpectralModel,
)
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, RingBackgroundMaker, FoVBackgroundMaker
from gammapy.modeling import Fit
from gammapy.maps import Map
from scipy.stats import norm
from gammapy.data import Observation
from gammapy.datasets import MapDataset, MapDatasetOnOff
from gammapy.estimators import ExcessMapEstimator, FluxPointsEstimator

In [3]:
import scipy.stats as stats
import math
import statistics
import os

In [4]:
os.environ['CALDB'] = '/home/rishank/anaconda2/envs/cta/share/caldb/'
!echo $CALDB
!ls $CALDB

/home/rishank/anaconda2/envs/cta/share/caldb/
data


In [5]:
irfs = load_cta_irfs(
    "$CALDB/data/cta/prod3b-v2/bcf/South_z20_50h/irf_file.fits"
)

Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [6]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 1.0 * u.hr
l = 0.05
pointing = SkyCoord(141.409467, 40.566962, unit="deg", frame="galactic")

In [7]:
# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(141.409467, 40.566962),
    binsz=0.02,
    width=(3, 3),
    frame="galactic",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_edges(
    np.logspace(-1.5, 1.5, 30), unit="TeV", name="energy_true", interp="log"
)

empty = MapDataset.create(geom, name="dataset-simu")

In [8]:
# Define sky model to used simulate the data.
# Here we use a Point spatial model and a Power Law spectral model.
spatial_model = PointSpatialModel(
    lon_0="141.409467 deg", lat_0="40.566962 deg", frame="galactic"
)
spectral_model = PowerLawSpectralModel(
    index=2.495, amplitude="2.72e-13 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_simu = SkyModel(
    spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="model-simu",
)

bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

models = Models([model_simu, bkg_model])
print(models)

Models

Component 0: SkyModel

  Name                      : model-simu
  Datasets names            : None
  Spectral model type       : PowerLawSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    index                   :   2.495              
    amplitude               :   2.72e-13  1 / (cm2 s TeV)
    reference    (frozen)   :   1.000  TeV         
    lon_0                   : 141.409  deg         
    lat_0                   :  40.567  deg         

Component 1: FoVBackgroundModel

  Name                      : dataset-simu-bkg
  Datasets names            : ['dataset-simu']
  Spectral model type       : PowerLawNormSpectralModel
  Parameters:
    norm                    :   1.000              
    tilt         (frozen)   :   0.000              
    reference    (frozen)   :   1.000  TeV         




In [9]:
def FoV(energy_reco, energy_true, livetime, irfs, pointing, geom, models):
    model = models.copy()
    obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
    maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
    maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=3.5 * u.deg)
    dataset = maker.run(empty, obs)
    dataset = maker_safe_mask.run(dataset, obs)
    dataset.models = model
    dataset.fake(random_state=42)
    
    dataset1 = dataset.copy()
    dataset1.models = bkg_model
    
    geom_image = geom.to_image().to_cube([energy_reco.squash()])
    regions = CircleSkyRegion(center=pointing, radius=0.2 * u.deg)
    exclusion_mask = Map.from_geom(geom_image)
    exclusion_mask.data = geom_image.region_mask([regions], inside=False)
    maker_fov = FoVBackgroundMaker(method="fit", exclusion_mask=exclusion_mask)
    
    ## original code
    dataset1 = maker_fov.run(dataset1)
    
    TS0 = dataset1.stat_sum()
    models_fit = model[0].copy()
    dataset1.models = models_fit
    fit = Fit([dataset1])
    result = fit.run()
    print(result)
    TS1 = dataset1.stat_sum()
    del_TS = TS0 - TS1
    
    return del_TS, dataset1, models_fit

In [10]:
def ring1(energy_reco, energy_true, livetime, irfs, pointing, geom, models):
    model = models.copy()
    obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
    maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
    maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=3.5 * u.deg)
    dataset = maker.run(empty, obs)
    dataset = maker_safe_mask.run(dataset, obs)
    dataset.models = model
    dataset.fake(random_state=42)
    
    geom_image = geom.to_image().to_cube([energy_reco.squash()])
    regions = CircleSkyRegion(center=pointing, radius=0.2 * u.deg)
    exclusion_mask = Map.from_geom(geom_image)
    exclusion_mask.data = geom_image.region_mask([regions], inside=False)
    ring_maker = RingBackgroundMaker(
        r_in="0.5 deg", width="0.3 deg", exclusion_mask=exclusion_mask
    )
    dataset_on_off = MapDatasetOnOff.create(
        geom=geom_image, energy_axis_true=energy_true, name="stacked"
    )
    dataset_on_off = ring_maker.run(dataset.to_image())
    
    TS0 = dataset_on_off.stat_sum()
    dataset_on_off.models = model
    fit = Fit([dataset_on_off])
    result = fit.run()
    print(result)
    TS1 = dataset_on_off.stat_sum()
    del_TS = TS0 - TS1
    
    return del_TS, dataset_on_off, model, dataset

In [11]:
def ring2(energy_reco, energy_true, livetime, irfs, pointing, geom, models):
    model = models.copy()
    obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
    maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
    maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=3.5 * u.deg)
    dataset = maker.run(empty, obs)
    dataset = maker_safe_mask.run(dataset, obs)
    dataset.models = model
    dataset.fake(random_state=2)
    
    geom_image = geom.to_image().to_cube([energy_reco.squash()])
    regions = CircleSkyRegion(center=pointing, radius=0.2 * u.deg)
    exclusion_mask = Map.from_geom(geom_image)
    exclusion_mask.data = geom_image.region_mask([regions], inside=False)
    ring_maker = RingBackgroundMaker(
        r_in="0.5 deg", width="0.3 deg", exclusion_mask=exclusion_mask
    )
    dataset = ring_maker.run(dataset) ## Instead of default dataset.to_image, only dataset is taken
    
    TS0 = dataset.stat_sum()
    dataset.models = model
    fit = Fit([dataset])
    result = fit.run()
    print(result)
    TS1 = dataset.stat_sum()
    del_TS = TS0 - TS1
    
    return del_TS, dataset, model

In [12]:
x = [25.0,50.0,100.0,200.0,500.0] * u.hr
y1 = []
y2 = []
y3 = []

In [13]:
for i in range(len(x)):
    y1.append(FoV(energy_reco, energy_true, x[i], irfs, pointing, geom, models))
    y2.append(ring1(energy_reco, energy_true, x[i], irfs, pointing, geom, model_simu))
    y3.append(ring2(energy_reco, energy_true, x[i], irfs, pointing, geom, model_simu))

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 110
	total stat : -1462994.32

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 104
	total stat : 22539.17

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 101
	total stat : 179921.73

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 140
	total stat : -4811556.19

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 106
	total stat : 22563.86

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	message    : Optimization terminated successfully.
	nfev    

RuntimeError: exception was raised in user function
User function arguments:
        par_000_index = +2.495000
    par_001_amplitude = +2.720000
        par_002_lon_0 = +1.414095
        par_003_lat_0 = +3.831743
Original python exception in user function:
ValueError: could not broadcast input array from shape (0) into shape (0,75)
  File "/home/rishank/anaconda2/envs/gammapy-0.18.2/lib/python3.7/site-packages/gammapy/modeling/iminuit.py", line 18, in fcn
    total_stat = self.function()
  File "/home/rishank/anaconda2/envs/gammapy-0.18.2/lib/python3.7/site-packages/gammapy/datasets/core.py", line 191, in stat_sum
    stat_sum += dataset.stat_sum()
  File "/home/rishank/anaconda2/envs/gammapy-0.18.2/lib/python3.7/site-packages/gammapy/datasets/map.py", line 1980, in stat_sum
    return Dataset.stat_sum(self)
  File "/home/rishank/anaconda2/envs/gammapy-0.18.2/lib/python3.7/site-packages/gammapy/datasets/core.py", line 55, in stat_sum
    stat = self.stat_array()
  File "/home/rishank/a

In [ ]:
del_TS_ = []
dataset_ = []
model_ = []
for i in range(len(x)):
    del_TS_.append(y1[i][0])
    dataset_.append(y1[i][1])
    model_.append(y1[i][2])
    
for i in range(len(x)):
    del_TS_.append(y2[i][0])
    dataset_.append(y2[i][3])
    model_.append(y2[i][2])
    
for i in range(len(x)):
    del_TS_.append(y3[i][0])
    dataset_.append(y3[i][1])
    model_.append(y3[i][2])

In [ ]:
print(del_TS_[0:5])
print(del_TS_[5:10])
print(del_TS_[10:15])

In [ ]:
plt.scatter(x,del_TS_[0:5])
#plt.xscale("log")
plt.legend(["FoV_Background"])
plt.ylabel("Del_TS")
plt.xlabel("LiveTime (hrs)")
plt.title("Original code")

In [ ]:
plt.scatter(x,del_TS_[5:10])
#plt.xscale("log")
plt.legend(["Ring_Background"])
plt.ylabel("Del_TS")
plt.xlabel("LiveTime (hrs)")
plt.title("Original code")

In [ ]:
plt.scatter(x,del_TS_[10:15])
plt.legend(["Ring_Background"])
plt.ylabel("Del_TS")
plt.xlabel("LiveTime (hrs)")
plt.title("Instead of default dataset.to_image, only dataset is taken")

In [ ]:
def flux_plt(model, dataset):
    spec = model.spectral_model
    energy_edges = np.logspace(-1.0, 1.0, 5) * u.TeV
    fpe = FluxPointsEstimator(energy_edges=energy_edges)
    flux_points = fpe.run(datasets=[dataset])
    energy_range = [0.1, 10] * u.TeV
    spec.plot(energy_range=energy_range, energy_power=2)
    ax = spec.plot_error(energy_range=energy_range, energy_power=2)
    flux_points.plot(ax=ax, energy_power=2)
    plt.title("Livetime = "+str(dataset.gti.time_delta.to(u.hr)))#, fontsize='15')
    return

In [ ]:
fig = plt.figure(figsize = [10,10], tight_layout=True)
fig.suptitle('FoV Background', verticalalignment = 'baseline', fontsize = 15)
plt.subplot(321)
flux_plt(model_[0],dataset_[0])
plt.subplot(322)
flux_plt(model_[1],dataset_[1])
plt.subplot(323)
flux_plt(model_[2],dataset_[2])
plt.subplot(324)
flux_plt(model_[3],dataset_[3])
plt.subplot(325)
flux_plt(model_[4],dataset_[4])

In [ ]:
fig = plt.figure(figsize = [10,10], tight_layout=True)
fig.suptitle('Ring Background 1', verticalalignment = 'baseline', fontsize = 15)
plt.subplot(321)
flux_plt(model_[5],dataset_[5])
plt.subplot(322)
flux_plt(model_[6],dataset_[6])
plt.subplot(323)
flux_plt(model_[7],dataset_[7])
plt.subplot(324)
flux_plt(model_[8],dataset_[8])
plt.subplot(325)
flux_plt(model_[9],dataset_[9])

In [ ]:
fig = plt.figure(figsize = [10,10], tight_layout=True)
fig.suptitle('Ring Background 2', verticalalignment = 'baseline', fontsize = 15)
plt.subplot(321)
flux_plt(model_[10],dataset_[10])
plt.subplot(322)
flux_plt(model_[11],dataset_[11])
plt.subplot(323)
flux_plt(model_[12],dataset_[12])
plt.subplot(324)
flux_plt(model_[13],dataset_[13])
plt.subplot(325)
flux_plt(model_[14],dataset_[14])